In [ ]:
# THIS CODE WAS DONE ON GOOGLE COLLAB

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import shutil
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from tqdm import tqdm



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!nvidia-smi

Tue May 13 03:56:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bert model

In [ ]:
huggingface_model='bert-base-uncased'
model=BertModel.from_pretrained(huggingface_model)
tokenizer=BertTokenizer.from_pretrained(huggingface_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### TASK 1

Some of the choices made:


1) [CLS] token emebdding was used at the beginning of the input sequence for classification tasks in BERT.


2) For tokenization, to ensure uniform input size padding was done to the max_length


3) Used sent_tokenize to process input text for extraction of individual sentence representations

In [ ]:
########################TASK 1###########################
########################TASK 1###########################
########################TASK 1###########################

import nltk
nltk.download('punkt_tab')
text_sent="""While we’ll be using two models, we will only train the logistic regression model. For DistillBERT, we’ll use a model that’s already pre-trained and has a grasp on the English language. This model, however is neither trained not fine-tuned to do sentence classification. We get some sentence classification capability, however, from the general objectives BERT is trained on. This is especially the case with BERT’s output for the first position (associated with the [CLS] token). I believe that’s due to BERT’s second training object – Next sentence classification. That objective seemingly trains the model to encapsulate a sentence-wide sense to the output at the first position."""

sentences=sent_tokenize(text_sent)
print('Original: ',sentences[0])
print('Tokenized: ',tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

# Figure out the maximum Length
max_len=0
for i in sentences:
    input_ids=tokenizer.encode(i,add_special_tokens=True)
    max_len=max(max_len,len(input_ids))

#print('The maximum length of the biggest sentence is ',max_len)

input_ids=[]
attention_masks=[]
# Tokenize the snetence, use [CLS] at the start and [SEP] at the end, map tokens to their IDS, pad sentence to the max_len,
for sent in sentences:
    encoded_dict=tokenizer.encode_plus(sent, add_special_tokens=True, max_length=max_len+1,padding='max_length',return_attention_mask=True,return_tensors='pt',truncation=True)

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

#Convert the lists to tensors
input_ids=torch.cat(input_ids,dim=0)
attention_masks=torch.cat(attention_masks,dim=0)
print('Original: ',sentences[0])
print('Token id view: ',input_ids[0])


# Put model in evaluation mode
model.eval()
# Process all sentences through BERT
with torch.no_grad():
    # Forward pass
    outputs = model(input_ids, attention_mask=attention_masks)
    # EMbeddings
    cls_embeddings = outputs.last_hidden_state[:, 0, :]

print("Shape of CLS embeddings:", cls_embeddings.shape)

# Example of using the CLS embeddings for the first sentence
print("Sentence CLS embedding:", cls_embeddings[0,:5])


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Original:  While we’ll be using two models, we will only train the logistic regression model.
Tokenized:  ['while', 'we', '’', 'll', 'be', 'using', 'two', 'models', ',', 'we', 'will', 'only', 'train', 'the', 'log', '##istic', 'regression', 'model', '.']
Token IDs:  [2096, 2057, 1521, 2222, 2022, 2478, 2048, 4275, 1010, 2057, 2097, 2069, 3345, 1996, 8833, 6553, 26237, 2944, 1012]
Original:  While we’ll be using two models, we will only train the logistic regression model.
Token id view:  tensor([  101,  2096,  2057,  1521,  2222,  2022,  2478,  2048,  4275,  1010,
         2057,  2097,  2069,  3345,  1996,  8833,  6553, 26237,  2944,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])
Shape of CLS embeddings: torch.Size([7, 768])
Sentence CLS embedding: tensor([-0.1391, -0.2947,  0.0603,  0.1204, -0.0222])


###LOAD THE DATASET FOR TASK A

In [ ]:
path="/content/drive/MyDrive/Colab Notebooks/Combined Data.csv"

df=pd.read_csv(path)
df.head()

limited_df = df.iloc[:15000].copy()
# Removing Unnamed: 0 column as it isn't needed
if 'Unnamed: 0' in limited_df.columns:
    limited_df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

# Split the limited dataset: first 700 rows for training, next 100 for testing
train_df = limited_df.iloc[:10500].reset_index(drop=True)
test_df = limited_df.iloc[11000:12000].reset_index(drop=True)
#train_df.head()
#test_df.head()


In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/train_data.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/test_data.csv', index=False)

In [ ]:
target_list = []
if 'status' in df.columns:
    target_list = df['status'].unique()
    print("\nUnique labels in the dataset:")
    print(target_list)
else:
    print("\nError: 'status' column not found in the DataFrame. Please specify the correct label column name.")



Unique labels in the dataset:
['Anxiety' 'Normal' 'Depression' 'Suicidal' 'Stress' 'Bipolar'
 'Personality disorder']


###LOAD THE DATASET FOR TASK B

In [ ]:
path_b="/content/drive/MyDrive/Colab Notebooks/Sentiment.csv"

try:
    # Try reading with utf-8 encoding
    df_b = pd.read_csv(path_b)
except UnicodeDecodeError:
    # try latin-1 encoding
    df_b = pd.read_csv(path_b, encoding='latin-1')

# Select only the text and sentiment columns
limited_df_b = df_b[['text', 'sentiment']].copy()
limited_df_b=limited_df_b[limited_df_b['sentiment'].notna()].copy()

print(limited_df_b.head())


# The numbers below for train_df_b and test_df_b can be changed if more training is needed
train_df_b = limited_df_b.iloc[:700].reset_index(drop=True)
test_df_b = limited_df_b.iloc[700:800].reset_index(drop=True)

print("\nFirst few rows of the training DataFrame:")
print(train_df_b.head())
print("\nFirst few rows of the testing DataFrame:")
print(test_df_b.head())

                                                text sentiment
0  Last session of the day  http://twitpic.com/67ezh   neutral
1   Shanghai is also really exciting (precisely -...  positive
2  Recession hit Veronique Branquinho, she has to...  negative
3                                        happy bday!  positive
4             http://twitpic.com/4w75p - I like it!!  positive

First few rows of the training DataFrame:
                                                text sentiment
0  Last session of the day  http://twitpic.com/67ezh   neutral
1   Shanghai is also really exciting (precisely -...  positive
2  Recession hit Veronique Branquinho, she has to...  negative
3                                        happy bday!  positive
4             http://twitpic.com/4w75p - I like it!!  positive

First few rows of the testing DataFrame:
                                                text sentiment
0   yeah, when you start reaching your early-mid-20s   neutral
1                           getti

In [ ]:
train_df_b.to_csv('/content/drive/MyDrive/Colab Notebooks/train_data_b.csv', index=False)
test_df_b.to_csv('/content/drive/MyDrive/Colab Notebooks/test_data_b.csv', index=False)

In [ ]:
target_list_b = []
if 'sentiment' in df_b.columns:
    target_list_b = df_b['sentiment'].unique()
    print("\nUnique labels in the dataset:")
    print(target_list_b)
    print(train_df_b.shape)
    print(test_df_b.shape)
else:
    print("\nError: 'sentiment' column is not there in the df_b")



Unique labels in the dataset:
['neutral' 'positive' 'negative' nan]
(700, 2)
(100, 2)


In [ ]:
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05
MAX_LEN = 256

###FOR DATASET

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, task):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.task = task

        # For Task A (Mental Health Classification)
        if self.task == 'A':
            if 'statement' not in df.columns:
                raise ValueError("Task A does not have  'statement' column")

            self.title = self.df['statement']
            # For one hot encoding
            self.target_list = ['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar', 'Personality disorder']
            self.targets = pd.get_dummies(self.df['status'], prefix='', prefix_sep='').reindex(
                columns=self.target_list, fill_value=0).values.astype(np.float32)

            # Initialize sentiment_targets as None for Task A
            self.sentiment_targets = np.zeros((len(self.df), 3), dtype=np.float32)

        # For Task B (Sentiment Analysis)
        elif self.task == 'B':
            if 'text' not in df.columns:
                raise ValueError("Task B does not have 'text' column")

            self.title = self.df['text']
            # For Task B, initialize targets as None
            self.targets = np.zeros((len(self.df), 7), dtype=np.float32)  # 7 is the number of mental health classes

            # For sentiment analysis preparation
            sentiment_list = ['negative', 'positive','neutral']
            # For one hot encoding
            self.sentiment_targets = pd.get_dummies(self.df['sentiment'], prefix='', prefix_sep='').reindex(
                columns=sentiment_list, fill_value=0).values.astype(np.float32)
        else:
            raise ValueError("Task must be either 'A' or 'B'")

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        #Uses BERT tokenixer to convert text into tokens by adding special tokens (like CLS, SEP), padding to max length, returning token ids as tensors, attention mask
        inputs = self.tokenizer.encode_plus(title, None, add_special_tokens=True, max_length=self.max_len,
                                            padding='max_length', return_token_type_ids=True, truncation=True,
                                            return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'sentiment_targets': torch.FloatTensor(self.sentiment_targets[index])
        }

In [ ]:
train_size = 0.8
####################### TASK A: Mental Health Classifcation ######################
# Create the validation split from the training data
train_dataset_full = train_df.copy()
# For stratified sampling to maintain class distribution
val_indices = []
for status in train_dataset_full['status'].unique():
    status_indices = train_dataset_full[train_dataset_full['status'] == status].index
    val_count = max(1, int(len(status_indices) * (1 - train_size)))
    val_indices.extend(np.random.choice(status_indices, val_count, replace=False))

# Create validation dataframe
val_df = train_dataset_full.loc[val_indices].reset_index(drop=True)
train_df = train_dataset_full.drop(val_indices).reset_index(drop=True)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")

train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN,task="A")
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN,task="A")

train_data_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=TRAIN_BATCH_SIZE, num_workers=0)
val_data_loader = torch.utils.data.DataLoader(valid_dataset, shuffle=False, batch_size=VALID_BATCH_SIZE, num_workers=0)

####################### TASK B: Sentiment Analysis ######################
# a valid validation split for Task B
train_dataset_full_B = train_df_b.copy()

#for stratified sampling to maintain class distribution for Task B
val_indices_B = []
for sentiment in train_dataset_full_B['sentiment'].unique():
    sentiment_indices = train_dataset_full_B[train_dataset_full_B['sentiment'] == sentiment].index
    val_count = max(1, int(len(sentiment_indices) * (1 - train_size)))
    val_indices_B.extend(np.random.choice(sentiment_indices, val_count, replace=False))

# Create validation dataframe for Task B
val_df_B = train_dataset_full_B.loc[val_indices_B].reset_index(drop=True)
train_df_B = train_dataset_full_B.drop(val_indices_B).reset_index(drop=True)

print("\nTask B - Sentiment Analysis:")
print(f"Training set size: {len(train_df_B)}")
print(f"Validation set size: {len(val_df_B)}")

# Create dataset objects for Task B
train_dataset_B = CustomDataset(train_df_B, tokenizer, MAX_LEN, task='B')
valid_dataset_B = CustomDataset(val_df_B, tokenizer, MAX_LEN, task='B')

# Create dataloaders for Task B
train_data_loader_B = torch.utils.data.DataLoader(train_dataset_B, shuffle=True, batch_size=TRAIN_BATCH_SIZE, num_workers=0)

val_data_loader_B = torch.utils.data.DataLoader(valid_dataset_B, shuffle=False, batch_size=VALID_BATCH_SIZE, num_workers=0)

Training set size: 8403
Validation set size: 2097

Task B - Sentiment Analysis:
Training set size: 561
Validation set size: 139


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device

device(type='cuda')

In [ ]:
# These numbers can be changed
task_a_weight = 0.5  # For task A.
task_b_weight = 0.5  # For for sentiment analysis task
class MultiTaskBERTModel(nn.Module):
    def __init__(self, model_name, num_classes_a=7, num_classes_b=3):
        """
        Multi-task BERT model for mental health classification and sentiment analysis

        Parameters:
        - model_name: The pretrained BERT model name (e.g., 'bert-base-uncased')
        - num_classes_a: Number of classes for Task A (mental health classification)
        - num_classes_b: Number of classes for Task B (sentiment analysis)
        """
        super(MultiTaskBERTModel, self).__init__()

        # Load pretrained BERT model
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)

        # Output layers for both tasks
        self.classifier_a = nn.Linear(768, num_classes_a)
        self.classifier_b = nn.Linear(768, num_classes_b)

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        Forward pass

        Parameters:
        - input_ids: Tokenized input IDs
        - attention_mask: Attention mask for padding
        - token_type_ids: Token type IDs

        Returns:
        - task_a_output: Output logits for Task A
        - task_b_output: Output logits for Task B
        """
        # Pass input through BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # Get the CLS token representation
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # Task-specific output layers
        task_a_output = self.classifier_a(pooled_output)
        task_b_output = self.classifier_b(pooled_output)

        return task_a_output, task_b_output

# Initializing the model
multi_task_model = MultiTaskBERTModel(huggingface_model).to(device)

# Initializing the optimizer
optimizer = torch.optim.AdamW(multi_task_model.parameters(), lr=LEARNING_RATE)


###################################### TASK 4 ####################################
def train_model(epochs, train_loader, val_loader, model, optimizer, task_a_weight=0.5, task_b_weight=0.5):
    """
    Training function for the multi-task model

    Parameters:
    - epochs: Number of training epochs
    - train_loader: DataLoader for training data
    - val_loader: DataLoader for validation data
    - model: The model to train
    - optimizer: The optimizer to use
    - task_a_weight: Weight for Task A loss
    - task_b_weight: Weight for Task B loss

    Returns:
    - a Trained model
    """
    # Binary Cross Entropy loss for multi-label classification
    task_a = nn.BCEWithLogitsLoss()
    task_b = nn.BCEWithLogitsLoss()

    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.train()
        total_loss = 0
        # For train dataset
        # for visuals
        progress_bar = tqdm(train_loader, desc="Training")
        for batch_idx, batch in enumerate(progress_bar):
            #Zeros out gradients
            optimizer.zero_grad() #
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            targets = batch['targets'].to(device)
            sentiment_targets = batch['sentiment_targets'].to(device)

            # Fforward pass to get outputs for both tasks
            task_a_output, task_b_output = model(input_ids, attention_mask, token_type_ids)

            # calculates loss for each task
            loss_a=task_a(task_a_output, targets)
            loss_b=task_b(task_b_output, sentiment_targets)

            #weighted loss
            loss=task_a_weight * loss_a + task_b_weight * loss_b

            #Backward Pass and optimize step
            loss.backward()
            optimizer.step()

            total_loss+=loss.item()
            progress_bar.set_postfix({'loss': total_loss/(batch_idx + 1)})

        # Calculate average loss at every epochs
        train_loss =total_loss/len(train_loader)
        history['train_loss'].append(train_loss)

        # For Validation dataset
        model.eval()
        val_loss = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                targets = batch['targets'].to(device)
                sentiment_targets = batch['sentiment_targets'].to(device)

                # Forward pass
                task_a_output, task_b_output = model(input_ids, attention_mask, token_type_ids)

                # Calculate loss
                loss_a = task_a(task_a_output, targets)
                loss_b = task_b(task_b_output, sentiment_targets)
                loss = task_a_weight * loss_a + task_b_weight * loss_b

                val_loss += loss.item()

        val_loss = val_loss / len(val_loader)
        history['val_loss'].append(val_loss)

        print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    return model


#Task 4

##Handling of Hypothetical Data
If the the data was not structured well enough, we can use several NLP techniques to filter and process the data. For example, the data processing pipeline could look like:

1) Filter the offensive words that are not desired from the training corpus.

2) Perform EDA to find out the vocabulary of the training corpus, frequency of certain words, and their corrlelation with the labels

3) Perform domain specific transformation on the training corpus that could help enchance the results. For example, modern LLMs tokenize numbers in a special manner for the LLM to be good at math

4) Choose a tokenizer to efficently tokenize the vocabulary.

## Forward Pass:
Model reaches saturation point With epochs=2.

LEARNING_RATE = 1e-05 is set to avoid vanishing Gradient  problem or exploding Gradient problem.

Since most sentences in my dataset are less than 256 words, MAX_LEN=256 preserves the entire context of the sentence

Batch size is set to 32 in order to preserve memory.

#Metrics:
Adam optimzer is used for stable training.

Categorical Cross Entropy is used loss since we have multi label classification task.


In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, multi_task_model, optimizer,
                          task_a_weight=task_a_weight, task_b_weight=task_b_weight)

# Function to make predictions
def predict_example(model, example_text, tokenizer, device, max_len=256):
    """
    Predict mental health class and sentiment for a given text
    """
    model.eval()
    # Clean the input text
    example_text = str(example_text).strip()

    encodings = tokenizer.encode_plus(
        example_text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)

        task_a_output, task_b_output = model(input_ids, attention_mask, token_type_ids)

        # Process Task A output (multi-label classification)
        task_a_probabilities = torch.sigmoid(task_a_output).cpu().numpy()[0]
        target_classes = ['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar', 'Personality disorder']
        predicted_class_a = target_classes[np.argmax(task_a_probabilities)]

        # Process Task B output (sentiment analysis)
        task_b_probabilities = torch.sigmoid(task_b_output).cpu().numpy()[0]
        sentiment_classes = ['Negative', 'Positive', 'Neutral']
        predicted_class_b = sentiment_classes[np.argmax(task_b_probabilities)]

        # Print results
        print("\n--- PREDICTION RESULTS ---")
        print(f"Input text: {example_text}")
        print("\nTask A: Mental Health Classification")
        print(f"Predicted class: {predicted_class_a}")

        print("\nTask B: Sentiment Analysis")
        print(f"Predicted sentiment: {predicted_class_b}")

        return {
            'task_a': {
                'predicted_class': predicted_class_a,
                'probabilities': {cls: float(task_a_probabilities[i]) for i, cls in enumerate(target_classes)}
            },
            'task_b': {
                'predicted_sentiment': predicted_class_b,
                'probabilities': {cls: float(task_b_probabilities[i]) for i, cls in enumerate(sentiment_classes)}
            }
        }

# Save the trained model
#torch.save(trained_model.state_dict(), 'mental_health_bert_model.pth')
#print("Model saved successfully.")

# Test prediction with an example
given_input = "I am feeling excited and full of happiness because I won a cricket match"

print(predict_example(trained_model, given_input, tokenizer, device))

Epoch 1/2


Validation: 100%|██████████| 66/66 [00:31<00:00,  2.09it/s]


Train Loss: 0.2351, Validation Loss: 0.1079
Epoch 2/2


Validation: 100%|██████████| 66/66 [00:31<00:00,  2.11it/s]

Train Loss: 0.0906, Validation Loss: 0.0685

--- PREDICTION RESULTS ---
Input text: I am feeling excited and full of happiness because I won a cricket match

Task A: Mental Health Classification
Predicted class: Normal

Task B: Sentiment Analysis
Predicted sentiment: Neutral
{'task_a': {'predicted_class': 'Normal', 'probabilities': {'Anxiety': 0.10141658782958984, 'Normal': 0.30692416429519653, 'Depression': 0.15447020530700684, 'Suicidal': 0.1671489030122757, 'Stress': 0.01926749013364315, 'Bipolar': 0.02004166692495346, 'Personality disorder': 0.017904745414853096}}, 'task_b': {'predicted_sentiment': 'Neutral', 'probabilities': {'Negative': 0.008344410918653011, 'Positive': 0.008437728509306908, 'Neutral': 0.009854908101260662}}}


#Task 3: Training Considerations

Discuss the implications and advantages of each scenario and explain your rationale as to how
the model should be trained given the following:

1. If the entire network should be frozen.

A1) When the entire network is frozen, it becomes an inference problem (i.e., predicting labels given data). No actual learning is done because there is no Backpropagation (i.e., no weights are updated).

2. If only the transformer backbone should be frozen.

A2) It is useful when dataset is small/medium sized and computational resouces are not available easily. If the transformer backbone is frozen, the network learns to adapt to the newly given data. In this case, it uses the transformer backbone to create very representative embeddings of the training data without having to train a large model. We usually attach feedforward head which learns to transform embeddings into label.

3. If only one of the task-specific heads (either for Task A or Task B) should be frozen.

A3) Freezing one of task specific heads can help modularize the end to end trained model. For example, if we have larger dataset for Task A, we can train a larger head while keeping others frozen to give better results for inference for TASK A. On the other hand, if we have smaller dataset for Task B, we can train smaller network and utilize other ML methods to train a better model for that task.



Consider a scenario where transfer learning can be beneficial. Explain how you would approach
the transfer learning process, including:
  1. The choice of a pre-trained model. ==>RoBERTa
  2. The layers you would freeze/unfreeze. ==> I would unfreeze Task heads, embedding layers, transformer layer.
  3. The rationale behind these choices. ==>:
  A3) RoBERTa is a BERT-based model that has been optimized for a range of NLP tasks, including those related to mental health, and has achieved state-of-the-art performance on several benchmarks. Assuming everything is frozen, I would unfreeze Task heads first becuase they need to be adjusted according to the specific task labels and train them (2-5 epochs) to reduce overfitting. We can also unfreeze embedding layer which is important for token represtentaions for topic related vocab.